# 板橋区議会発言者統計分析
# Itabashi Ward Council Speaker Statistics Analysis

このノートブックでは、板橋区議会の発言者に関する統計分析を行います。

## 分析内容
1. 発言者別の発言回数・文字数統計
2. 発言者の関心分野分析
3. 発言パターンの分析
4. 委員会別の発言者活動度

In [ ]:
# 必要なライブラリのインポート
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# データ読み込み
def load_minutes_data(file_path):
    """会議録JSONデータを読み込む"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

try:
    minutes_data = load_minutes_data('../crawler/sample/sample_minutes.json')
    print(f"読み込み完了: {len(minutes_data)} 件の会議録")
except FileNotFoundError:
    print("サンプルデータが見つかりません。")
    minutes_data = []

In [ ]:
# 発言者データの構造化
class SpeakerAnalyzer:
    def __init__(self):
        self.policy_keywords = {
            '教育': ['教育', '学校', '児童', '生徒', '授業', '先生', '教師', '学習'],
            '福祉': ['福祉', '高齢者', '介護', '年金', '障害', '子育て', '保育'],
            '環境': ['環境', 'ゴミ', '清掃', 'リサイクル', '公園', '緑化'],
            '防災': ['防災', '地震', '火災', '避難', '消防', '安全'],
            '交通': ['交通', '道路', '駅', 'バス', '駐車場', '歩道'],
            '予算': ['予算', '財政', '税収', '支出', '決算', '補正予算']
        }
    
    def extract_speaker_data(self, minutes_data):
        """発言者データを抽出"""
        speaker_speeches = []
        
        for meeting in minutes_data:
            meeting_date = meeting.get('meeting_date', '')
            committee = meeting.get('committee', '')
            
            if 'agenda_items' in meeting:
                for item in meeting['agenda_items']:
                    speaker = item.get('speaker', '').strip()
                    speech_text = item.get('speech_text', '')
                    
                    if speaker and speech_text:
                        # 発言者名の正規化
                        normalized_speaker = self.normalize_speaker_name(speaker)
                        
                        speech_data = {
                            'speaker': normalized_speaker,
                            'original_speaker': speaker,
                            'speech_text': speech_text,
                            'speech_length': len(speech_text),
                            'word_count': len(speech_text.split()),
                            'meeting_date': meeting_date,
                            'committee': committee,
                            'agenda_item': item.get('agenda_item', ''),
                            'policy_interests': self.identify_policy_interests(speech_text)
                        }
                        speaker_speeches.append(speech_data)
        
        return speaker_speeches
    
    def normalize_speaker_name(self, speaker):
        """発言者名の正規化"""
        # 一般的な敬称や役職を除去
        speaker = re.sub(r'(議員|委員|議長|委員長|副委員長|さん)$', '', speaker)
        speaker = speaker.strip()
        return speaker if speaker else '不明'
    
    def identify_policy_interests(self, speech_text):
        """発言内容から政策関心分野を特定"""
        interests = []
        
        for policy, keywords in self.policy_keywords.items():
            if any(keyword in speech_text for keyword in keywords):
                interests.append(policy)
        
        return interests

# 発言者分析の実行
analyzer = SpeakerAnalyzer()
speaker_data = analyzer.extract_speaker_data(minutes_data)

if speaker_data:
    df_speakers = pd.DataFrame(speaker_data)
    df_speakers['meeting_date'] = pd.to_datetime(df_speakers['meeting_date'], errors='coerce')
    print(f"発言データ数: {len(df_speakers)}")
    print(f"発言者数: {df_speakers['speaker'].nunique()}")
else:
    print("発言者データがありません")
    df_speakers = pd.DataFrame()

In [ ]:
# 発言者別統計分析
if not df_speakers.empty:
    # 発言者別集計
    speaker_stats = df_speakers.groupby('speaker').agg({
        'speech_text': 'count',           # 発言回数
        'speech_length': ['sum', 'mean'], # 総文字数、平均文字数
        'word_count': ['sum', 'mean'],    # 総単語数、平均単語数
        'meeting_date': ['min', 'max']    # 初回・最終発言日
    }).round(1)
    
    # 列名を整理
    speaker_stats.columns = [
        'speech_count', 'total_chars', 'avg_chars', 
        'total_words', 'avg_words', 'first_speech', 'last_speech'
    ]
    
    # 発言者の活動度でソート
    speaker_stats = speaker_stats.sort_values('speech_count', ascending=False)
    
    print("=== 発言者別統計（上位10名） ===")
    print(speaker_stats.head(10))
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 発言回数トップ10
    top_speakers = speaker_stats.head(10)
    axes[0,0].barh(range(len(top_speakers)), top_speakers['speech_count'])
    axes[0,0].set_yticks(range(len(top_speakers)))
    axes[0,0].set_yticklabels(top_speakers.index)
    axes[0,0].set_title('Top 10 Speakers by Speech Count')
    axes[0,0].invert_yaxis()
    
    # 2. 総文字数トップ10
    top_by_chars = speaker_stats.nlargest(10, 'total_chars')
    axes[0,1].barh(range(len(top_by_chars)), top_by_chars['total_chars'])
    axes[0,1].set_yticks(range(len(top_by_chars)))
    axes[0,1].set_yticklabels(top_by_chars.index)
    axes[0,1].set_title('Top 10 Speakers by Total Characters')
    axes[0,1].invert_yaxis()
    
    # 3. 発言回数の分布
    axes[1,0].hist(speaker_stats['speech_count'], bins=20, alpha=0.7)
    axes[1,0].set_title('Distribution of Speech Counts')
    axes[1,0].set_xlabel('Speech Count')
    axes[1,0].set_ylabel('Number of Speakers')
    
    # 4. 平均文字数 vs 発言回数の散布図
    axes[1,1].scatter(speaker_stats['speech_count'], speaker_stats['avg_chars'], alpha=0.6)
    axes[1,1].set_title('Speech Count vs Average Characters')
    axes[1,1].set_xlabel('Speech Count')
    axes[1,1].set_ylabel('Average Characters per Speech')
    
    plt.tight_layout()
    plt.show()
else:
    print("統計分析するデータがありません")

In [ ]:
# 政策関心分野分析
if not df_speakers.empty:
    # 各発言者の政策関心を集計
    speaker_interests = defaultdict(lambda: defaultdict(int))
    
    for _, row in df_speakers.iterrows():
        speaker = row['speaker']
        interests = row['policy_interests']
        
        for interest in interests:
            speaker_interests[speaker][interest] += 1
    
    # 上位発言者の関心分野をヒートマップで可視化
    top_speakers_list = speaker_stats.head(8).index.tolist()
    policy_areas = ['教育', '福祉', '環境', '防災', '交通', '予算']
    
    heatmap_data = []
    for speaker in top_speakers_list:
        row = [speaker_interests[speaker][policy] for policy in policy_areas]
        heatmap_data.append(row)
    
    if heatmap_data:
        heatmap_df = pd.DataFrame(heatmap_data, 
                                 index=top_speakers_list, 
                                 columns=policy_areas)
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(heatmap_df, annot=True, cmap='YlOrRd', fmt='d')
        plt.title('Policy Interest Heatmap (Top Speakers)')
        plt.ylabel('Speakers')
        plt.xlabel('Policy Areas')
        plt.tight_layout()
        plt.show()
    
    # 政策分野別の専門性スコア計算
    policy_specialists = defaultdict(list)
    
    for speaker in speaker_interests:
        total_speeches = sum(speaker_interests[speaker].values())
        if total_speeches >= 2:  # 最低2回以上発言
            for policy, count in speaker_interests[speaker].items():
                specialization_score = count / total_speeches
                if specialization_score >= 0.3:  # 30%以上がその分野
                    policy_specialists[policy].append(
                        (speaker, specialization_score, count)
                    )
    
    print("\n=== 政策分野別専門家 ===")
    for policy, specialists in policy_specialists.items():
        if specialists:
            specialists.sort(key=lambda x: x[1], reverse=True)
            print(f"\n{policy}:")
            for speaker, score, count in specialists[:3]:
                print(f"  {speaker}: {score:.1%} ({count}回)")
else:
    print("政策関心分析するデータがありません")

In [ ]:
# 委員会別発言者活動度
if not df_speakers.empty:
    # 委員会別の発言統計
    committee_stats = df_speakers.groupby(['committee', 'speaker']).agg({
        'speech_text': 'count',
        'speech_length': 'sum'
    }).rename(columns={'speech_text': 'speech_count', 'speech_length': 'total_chars'})
    
    # 委員会別の上位発言者
    print("\n=== 委員会別上位発言者 ===")
    for committee in df_speakers['committee'].unique():
        if pd.notna(committee) and committee:
            committee_data = committee_stats.loc[committee]
            top_in_committee = committee_data.nlargest(3, 'speech_count')
            
            print(f"\n{committee}:")
            for speaker, data in top_in_committee.iterrows():
                print(f"  {speaker}: {data['speech_count']}回 ({data['total_chars']}文字)")
    
    # 発言パターン分析
    if df_speakers['meeting_date'].notna().sum() > 0:
        # 月別発言活動
        df_with_date = df_speakers[df_speakers['meeting_date'].notna()].copy()
        df_with_date['year_month'] = df_with_date['meeting_date'].dt.to_period('M')
        
        monthly_activity = df_with_date.groupby(['year_month', 'speaker']).size().unstack(fill_value=0)
        
        # 上位5名の月別活動をプロット
        if not monthly_activity.empty:
            top_5_speakers = speaker_stats.head(5).index
            available_speakers = [s for s in top_5_speakers if s in monthly_activity.columns]
            
            if available_speakers:
                plt.figure(figsize=(12, 6))
                for speaker in available_speakers:
                    plt.plot(monthly_activity.index, monthly_activity[speaker], 
                           marker='o', label=speaker)
                
                plt.title('Monthly Speech Activity (Top 5 Speakers)')
                plt.xlabel('Year-Month')
                plt.ylabel('Number of Speeches')
                plt.legend()
                plt.xticks(rotation=45)
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
else:
    print("委員会別分析するデータがありません")

In [ ]:
# 結果のエクスポート
if not df_speakers.empty:
    # 発言者統計をCSVで出力
    speaker_stats.to_csv('speaker_statistics.csv', encoding='utf-8')
    
    # 政策関心マトリックスをCSVで出力
    if 'heatmap_df' in locals():
        heatmap_df.to_csv('policy_interest_matrix.csv', encoding='utf-8')
    
    print("\n=== エクスポート完了 ===")
    print("- speaker_statistics.csv: 発言者別統計")
    print("- policy_interest_matrix.csv: 政策関心マトリックス")
    
    # サマリー
    print("\n=== 分析サマリー ===")
    print(f"総発言数: {len(df_speakers)}")
    print(f"発言者数: {df_speakers['speaker'].nunique()}")
    print(f"委員会数: {df_speakers['committee'].nunique()}")
    print(f"平均発言文字数: {df_speakers['speech_length'].mean():.0f}文字")
    print(f"最も活発な発言者: {speaker_stats.index[0]} ({speaker_stats.iloc[0]['speech_count']}回)")
else:
    print("エクスポートするデータがありません")

## 分析結果のまとめ

このノートブックでは以下の発言者統計分析を実行しました：

1. **基本統計**: 発言回数、文字数、単語数の集計
2. **活動度分析**: 最も活発な発言者の特定
3. **政策関心分析**: 各発言者の専門分野の特定
4. **委員会別活動**: 委員会ごとの発言パターン
5. **時系列分析**: 発言者の時間的活動パターン

### 得られる知見
- 議会での発言頻度の分布
- 各議員の専門分野や関心領域
- 委員会での役割分担
- 発言の質（長さ、内容の専門性）

### 今後の拡張案
- 感情分析による発言トーンの分析
- ネットワーク分析による議員間の関係性
- 質問と答弁のペア分析
- 政党別の発言傾向分析
- 新人議員と経験豊富な議員の比較